In [3]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_768/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/22 16:01:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/22 16:01:29 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [71]:
import pandas as pd

# Sample data for Heights table
heights_data = {
    'id': list(range(1, 13)),
    'height': [0, 1, 0, 2, 1, 0, 1, 3, 2, 1, 2, 1]
}

# Create DataFrame for Heights
heights_df = pd.DataFrame(heights_data)

# Display the DataFrame
print("Heights table:")
print(heights_df)



df_person = spark.createDataFrame(heights_df)
df_person.createOrReplaceTempView("Heights")

import pandas as pd

# Sample data for Heights table
heights_data = {
    'id':     [1, 2, 3, 4, 5, 6, 7, 8],
    'height': [0, 1, 2, 3, 0, 1, 0, 5]
}

# Create DataFrame for Heights
heights_df = pd.DataFrame(heights_data)

# Display the DataFrame
print("Heights_1 table:")
print(heights_df)



df_person = spark.createDataFrame(heights_df)
df_person.createOrReplaceTempView("Heights_1")

import pandas as pd

# Sample data for Heights table
heights_data = {
    'id':     list(range(1, 13)),
    'height': [0, 1, 0, 2, 1, 0, 1, 2, 2, 2, 2, 2]
}

# Create DataFrame for Heights
heights_df = pd.DataFrame(heights_data)

# Display the DataFrame
print("Heights_1 table:")
print(heights_df)



df_person = spark.createDataFrame(heights_df)
df_person.createOrReplaceTempView("Heights_2")



Heights table:
    id  height
0    1       0
1    2       1
2    3       0
3    4       2
4    5       1
5    6       0
6    7       1
7    8       3
8    9       2
9   10       1
10  11       2
11  12       1
Heights_1 table:
   id  height
0   1       0
1   2       1
2   3       2
3   4       3
4   5       0
5   6       1
6   7       0
7   8       5
Heights_1 table:
    id  height
0    1       0
1    2       1
2    3       0
3    4       2
4    5       1
5    6       0
6    7       1
7    8       2
8    9       2
9   10       2
10  11       2
11  12       2


In [72]:
table_name = "Heights_2"

In [73]:
query_1 = f"""
    with bound_height as (
        select
            h.id as start_id,
            h.height as start_height,
            h1.id as end_id,
            h1.height as end_height
        from {table_name} as h left join {table_name} h1 on h.id < h1.id - 1
        where h1.id is not null
        order by h.id
    ),
    
    range_height as (
        select
            b.*,
            h.height as between_height,
            (h.height < b.start_height and h.height < b.end_height) as is_less_than_bound
        from bound_height as b left join {table_name} as h on h.id > b.start_id and h.id < b.end_id
    ),
    
    bound as (
        select
            *
        from bound_height as b where (b.start_id, b.end_id) not in (
            select start_id, end_id from range_height where  not is_less_than_bound
        )
    ),
    
    satisfy_bound as (
        select
            s.*
        from bound s left join bound s1 
            on  s.start_id >= s1.start_id
            and s.end_id <= s1.end_id
            and pow(s.start_id - s1.start_id, 2) + pow(s.end_id - s1.end_id, 2) != 0
        where s1.start_id is null
    )
    
    select 
        sum(remain_height) total_trapped_water 
    from (
        select
            s.*,
            h.*,
            least(s.start_height, s.end_height) - h.height as remain_height
        from satisfy_bound s left join {table_name} h 
            on h.id > s.start_id and h.id < s.end_id
    )

"""

In [74]:
spark.sql(query_1).show(700)

+-------------------+
|total_trapped_water|
+-------------------+
|                  5|
+-------------------+

